# Prediction of BitCoin price with LSTM network

## Intro

Main purpose of this project is to create Long short-term memory (LSTM) neural network for prediction of Bitcoin close price. 

## Import necessary libraries

In [122]:
# general libs
from datetime import datetime
import os
import glob
from itertools import cycle

# data analysis libs
import pandas as pd
import numpy as np
import math
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots


#ML
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score, r2_score
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM




## Read historical data

Data was downloaded from  [Binance.com](https://data.binance.vision/data/spot/daily/klines/BTCTUSD/15m/)

In [59]:
# define names of columns because .csv files dont contain column names
col_names=['Open_time','Open','High','Low','Close','Volume','Close_time','Quote_asset_volume','Number_trades','Taker_buy_base_asset_volume','Taker_buy_quote_asset_volume','Ignore']
all_files = glob.glob("*.csv")
# .csv files downloaded from binance.com, and contains information about pair BTC-USDT with frequency 15 minutes from 21.0.22 to 23.09.22
# load multiple .csv files to dataframe
df = pd.concat((pd.read_csv(f,  names=col_names, header=None) for f in all_files))
# remove irrelevant column from dataset
df.drop('Ignore',axis=1, inplace=True)

print('Total number of rows present in the dataset: ',df.shape[0])
print('Total number of fields present in the dataset: ',df.shape[1])


Total number of rows present in the dataset:  3264
Total number of fields present in the dataset:  11


Final data frame has the following look:

In [60]:
df

,Open_time,Open,High,Low,Close,Volume,Close_time,Quote_asset_volume,Number_trades,Taker_buy_base_asset_volume,Taker_buy_quote_asset_volume
0,1661040000000,21117.63,21175.40,21053.00,21175.40,0.43148,1661040899999,9098.699570,16,0.05805,1227.958536
1,1661040900000,21270.68,21303.78,21163.95,21216.44,0.34454,1661041799999,7321.880707,9,0.23647,5028.601272
2,1661041800000,21235.34,21235.34,21213.40,21213.40,0.06032,1661042699999,1280.117826,4,0.00000,0.000000
3,1661042700000,21193.23,21193.23,21172.76,21178.23,0.04794,1661043599999,1015.309446,4,0.04794,1015.309446
4,1661043600000,21199.58,21199.58,21198.41,21198.41,0.03110,1661044499999,659.278799,2,0.02405,509.821760
...,...,...,...,...,...,...,...,...,...,...,...
91,1663973100000,19253.60,19353.43,19253.60,19353.43,0.11924,1663973999999,2299.246998,8,0.09424,1816.840748
92,1663974000000,19373.70,19388.94,19360.22,19360.22,0.08288,1663974899999,1606.155661,3,0.08288,1606.155661
93,1663974900000,19394.42,19397.66,19328.34,19351.94,0.08373,1663975799999,1621.439326,9,0.00532,103.111573
94,1663975800000,19393.06,19393.06,19330.74,19334.07,0.32912,1663976699999,6367.233436,17,0.07476,1447.324304


And can be represented as popular candlestick chart

In [127]:
# create graphic object (go)
fig = go.Figure(go.Candlestick(
        x=pd.to_datetime(df['Close_time'], unit='ms'),
        open=df['Open'],
        high=df['High'],
        low=df['Low'],
        close=df['Close']))

fig.update_layout(
    title='BTC Candlestick',
    yaxis_title='USDT',
    xaxis_title='Date',
    plot_bgcolor='white', 
    font_size=15, 
    font_color='black'
)
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

Or as a graph of close price, because exactly this part is important to us.

In [62]:
fig = px.line(df,x=pd.to_datetime(df['Close_time'], unit='ms'), y=df['Close'],
              labels={'Close':'USDT','x': 'Date'})
fig.update_layout(title_text='Сlose price for BTC-USDT pair',
                  plot_bgcolor='white', font_size=15, font_color='black', legend_title_text='Close Price')
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

## Data preparation

### [Chaikin Money Flow](https://www.chaikinanalytics.com/chaikin-money-flow/)
Chaikin Money Flow (the CMF Indicator) was created by Marc Chaikin in the early 1980s as a way to measure the accumulation (buying pressure) vs. distribution (selling pressure) of a stock over time.

Chaikin Money Flow is displayed as a oscillator around a 0-line, ranging from +100 to -100, with green regions representing net buying pressure, and red regions representing net selling pressure.

Because of the way the calculation was designed, money flow persistency, can be signs of accumulation or distribution by major institutions which have a disproportionate effect on future price movement. 
 
This is true even though the calculation looks only at price/volume activity, rather than directly monitoring institutional order flow.

A short-term money flow divergence — where price rises but Money Flow stays negative, or vice versa—can indicate an increased risk of a short-term reversal.

In [121]:
rang = 8
df["Money Flow"]=round(((df['Close'] - df['Low']) - (df['High'] - df['Close']))/ (df['High'] - df['Low']) * df['Volume'],2)
df.loc[df['Money Flow'].isnull(),['Money Flow']]=0
df['CMF']=df['Money Flow'].rolling(window = rang).sum()/df['Volume'].rolling(window = rang).sum()
df.loc[df['CMF'].isnull(),['CMF']]=0

fig = px.line(plotdf,x=pd.to_datetime(df['Close_time'], unit='ms'), y=df['CMF'],
              labels={'y':'%','x': 'Date'})
fig.update_layout(title_text='Chaikin Money Flow for BTC-USDT pair',
                  plot_bgcolor='white', font_size=15, font_color='black', legend_title_text='Close Price')
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()


### [The relative strength index](https://en.wikipedia.org/wiki/Relative_strength_index)

The relative strength index (RSI) is a momentum indicator used in technical analysis. RSI measures the speed and magnitude of a security's recent price changes to evaluate overvalued or undervalued conditions in the price of that security.

The RSI is displayed as an oscillator (a line graph) on a scale of zero to 100. The indicator was developed by J. Welles Wilder Jr. and introduced in his seminal 1978 book, New Concepts in Technical Trading Systems.1

The RSI can do more than point to overbought and oversold securities. It can also indicate securities that may be primed for a trend reversal or corrective pullback in price. It can signal when to buy and sell. Traditionally, an RSI reading of 70 or above indicates an overbought situation. A reading of 30 or below indicates an oversold condition.


In [120]:
df['Delta price']=df['Close'].diff()
df.loc[df['Delta price'].isnull(),['Delta price']]=0

up = df['Delta price'].clip(lower=0)
down = -1 * df['Delta price'].clip(upper=0)
ma_up = up.rolling(window = rang).mean()

ma_down = down.rolling(window = rang).mean()
rsi = ma_up / ma_down
rsi = 1 - (1/(1 + rsi))
df['RSI']=rsi
df.loc[df['RSI'].isnull(),['RSI']]=0



fig = px.line(plotdf,x=pd.to_datetime(df['Close_time'], unit='ms'), y=df['RSI'],
              labels={'y':'%','x': 'Date'})
fig.update_layout(title_text='Comparision between original close price vs predicted close price',
                  plot_bgcolor='white', font_size=15, font_color='black', legend_title_text='Close Price')
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()


RSI and CMF are two very popular technical indicators used in the analysis of financial markets, because of that fact we will be use them togethe with close value and volume as features for close price prediction for the previous eight steps (four hours).


In [123]:
df_sub = df.iloc[rang:]
X=df_sub[['CMF','RSI', 'Volume', 'Close']].values
for_plot=(np.array(df_sub['Close']).reshape(-1,1))
print(for_plot.shape)
Y=df_sub['Close'].values
print(X.shape); print(Y.shape)

(3256, 1)
(3256, 4)
(3256,)


In [124]:
def create_dataset(dataset, time_step=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-time_step):
        a = dataset[i:(i+time_step), :]   ###i=0, 0,1,2,3-----99   100 
        dataX.append(a)
        dataY.append(dataset[i + time_step, -1])
    return np.array(dataX), np.array(dataY)



time_step = 8

data_split=0.8
test_size=int(df_sub.shape[0]*data_split)
scaler=MinMaxScaler(feature_range=(0,1))
ys=scaler.fit_transform(Y.reshape(-1,1))
#X = scaler.fit_transform(X.reshape(-1,4))
X[:,3]=ys.ravel()
totrain = X[:test_size,:]
tovalid = X[test_size:,:]
#Yf=scaler.fit_transform(Y.reshape(-1,1))
X_train, y_train = create_dataset(totrain, time_step)
X_test, y_test = create_dataset(tovalid, time_step)

print("X_train: ", X_train.shape)
print("y_train: ", y_train.shape)
print("X_test: ", X_test.shape)
print("y_test", y_test.shape)

X_train:  (2596, 8, 4)
y_train:  (2596,)
X_test:  (644, 8, 4)
y_test (644,)


In [97]:
model=Sequential()

model.add(LSTM(30,input_shape=(None,4),activation="relu"))

model.add(Dense(1))

model.compile(loss="mean_squared_error",optimizer="adam")

In [98]:
history = model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=200,batch_size=32,verbose=1)

Epoch 1/200
82/82 [==============================] - 1s 5ms/step - loss: 0.0982 - val_loss: 0.0542
Epoch 2/200
82/82 [==============================] - 0s 3ms/step - loss: 0.0051 - val_loss: 0.0057
Epoch 3/200
82/82 [==============================] - 0s 3ms/step - loss: 0.0015 - val_loss: 0.0046
Epoch 4/200
82/82 [==============================] - 0s 3ms/step - loss: 0.0012 - val_loss: 0.0038
Epoch 5/200
82/82 [==============================] - 0s 3ms/step - loss: 9.9186e-04 - val_loss: 0.0025
Epoch 6/200
82/82 [==============================] - 0s 3ms/step - loss: 7.5855e-04 - val_loss: 0.0017
Epoch 7/200
82/82 [==============================] - 0s 3ms/step - loss: 6.2284e-04 - val_loss: 0.0011
Epoch 8/200
82/82 [==============================] - 0s 3ms/step - loss: 5.5302e-04 - val_loss: 9.4987e-04
Epoch 9/200
82/82 [==============================] - 0s 3ms/step - loss: 4.9639e-04 - val_loss: 9.2559e-04
Epoch 10/200
82/82 [==============================] - 0s 3ms/step - loss: 4.8349e

82/82 [==============================] - 0s 3ms/step - loss: 2.4731e-04 - val_loss: 5.2688e-04
Epoch 78/200
82/82 [==============================] - 0s 3ms/step - loss: 2.4744e-04 - val_loss: 4.8047e-04
Epoch 79/200
82/82 [==============================] - 0s 3ms/step - loss: 2.4494e-04 - val_loss: 4.9664e-04
Epoch 80/200
82/82 [==============================] - 0s 3ms/step - loss: 2.8574e-04 - val_loss: 5.0102e-04
Epoch 81/200
82/82 [==============================] - 0s 3ms/step - loss: 2.4608e-04 - val_loss: 6.4411e-04
Epoch 82/200
82/82 [==============================] - 0s 3ms/step - loss: 2.7169e-04 - val_loss: 4.8032e-04
Epoch 83/200
82/82 [==============================] - 0s 3ms/step - loss: 2.4041e-04 - val_loss: 5.0207e-04
Epoch 84/200
82/82 [==============================] - 0s 3ms/step - loss: 2.4057e-04 - val_loss: 4.7342e-04
Epoch 85/200
82/82 [==============================] - 0s 3ms/step - loss: 2.3579e-04 - val_loss: 4.8554e-04
Epoch 86/200
82/82 [=====================

82/82 [==============================] - 0s 3ms/step - loss: 2.1259e-04 - val_loss: 4.7861e-04
Epoch 153/200
82/82 [==============================] - 0s 3ms/step - loss: 2.1927e-04 - val_loss: 4.6249e-04
Epoch 154/200
82/82 [==============================] - 0s 3ms/step - loss: 1.9604e-04 - val_loss: 5.1258e-04
Epoch 155/200
82/82 [==============================] - 0s 3ms/step - loss: 1.9992e-04 - val_loss: 4.9172e-04
Epoch 156/200
82/82 [==============================] - 0s 3ms/step - loss: 1.9695e-04 - val_loss: 4.9478e-04
Epoch 157/200
82/82 [==============================] - 0s 3ms/step - loss: 1.9337e-04 - val_loss: 4.5525e-04
Epoch 158/200
82/82 [==============================] - 0s 3ms/step - loss: 1.9162e-04 - val_loss: 4.6776e-04
Epoch 159/200
82/82 [==============================] - 0s 3ms/step - loss: 1.9043e-04 - val_loss: 4.8850e-04
Epoch 160/200
82/82 [==============================] - 0s 3ms/step - loss: 2.0476e-04 - val_loss: 5.2211e-04
Epoch 161/200
82/82 [============

In [99]:
train_predict=model.predict(X_train)
test_predict=model.predict(X_test)
train_predict.shape, test_predict.shape

21/21 [==============================] - 0s 1ms/step


((2596, 1), (644, 1))

In [100]:
train_predict = scaler.inverse_transform(train_predict)
test_predict = scaler.inverse_transform(test_predict)
original_ytrain = scaler.inverse_transform(y_train.reshape(-1,1)) 
original_ytest = scaler.inverse_transform(y_test.reshape(-1,1)) 


# Evaluation metrices RMSE and MAE
print("Train data RMSE: ", math.sqrt(mean_squared_error(original_ytrain,train_predict)))
print("Train data MSE: ", mean_squared_error(original_ytrain,train_predict))
print("Train data MAE: ", mean_absolute_error(original_ytrain,train_predict))
print("-------------------------------------------------------------------------------------")
print("Test data RMSE: ", math.sqrt(mean_squared_error(original_ytest,test_predict)))
print("Test data MSE: ", mean_squared_error(original_ytest,test_predict))
print("Test data MAE: ", mean_absolute_error(original_ytest,test_predict))

print("Test data R2 score:", r2_score(original_ytest, test_predict))
#original_ytest, test_predict

Train data RMSE:  58.370773684264826
Train data MSE:  3407.147220499663
Train data MAE:  40.86666033681625
-------------------------------------------------------------------------------------
Test data RMSE:  96.57270874859128
Test data MSE:  9326.28807504024
Test data MAE:  61.359320166925436
Test data R2 score: 0.9565673503703552


In [101]:
# shift train predictions for plotting

look_back=time_step
trainPredictPlot = np.empty_like(for_plot)
print(for_plot.shape)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[look_back:len(train_predict)+look_back, :] = train_predict
print("Train predicted data: ", trainPredictPlot.shape)

# shift test predictions for plotting
testPredictPlot = np.empty_like(for_plot)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(train_predict)+(look_back*2):len(for_plot), :] = test_predict
print("Test predicted data: ", testPredictPlot.shape)

names = cycle(['Original close price','Train predicted close price','Test predicted close price'])


plotdf = pd.DataFrame({'date': pd.to_datetime(df_sub['Close_time'], unit='ms'),
                       'original_close': df_sub['Close'],
                      'train_predicted_close': trainPredictPlot.reshape(1,-1)[0].tolist(),
                      'test_predicted_close': testPredictPlot.reshape(1,-1)[0].tolist()})

fig = px.line(plotdf,x=plotdf['date'], y=[plotdf['original_close'],plotdf['train_predicted_close'],
                                          plotdf['test_predicted_close']],
              labels={'value':'USDT','date': 'Date'})
fig.update_layout(title_text='Comparision between original close price vs predicted close price',
                  plot_bgcolor='white', font_size=15, font_color='black', legend_title_text='Close Price')
fig.for_each_trace(lambda t:  t.update(name = next(names)))

fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

(3256, 1)
Train predicted data:  (3256, 1)
Test predicted data:  (3256, 1)


## Conclusion

Designed LSTM model unfortunately can't predict price change , network react only after price cnhange, that fact can be seen from comparsion chart. The reasons that may have caused this result are many. Some of then may be:

 - Only the historical data of closing prices, RSI, CMF and Volume are not enough to predict the stock price behaviour
 - The model could be improved